In [15]:
import pacepredictor as pp
import importlib
import plotly.express as px
import pandas as pd
import numpy as np

importlib.reload(pp)

<module 'pacepredictor' from '/home/adam/Concept2-Pace-Predictor/pacepredictor.py'>

# Load Data

In [16]:
file_path = "./"
file_5_2_10 = "5_2_10.csv"

pp_5_2_10 = pp.Predictor(f"{file_path}{file_5_2_10}", random_state=42)
pp_5_2_10.df

,time_5000,time_2000,time_10000,adaptive_rowing_category,age,height,weight,weight_class,profile_id
4,1086.8,416.4,2222.6,able,56,188.0,97.1,H,83
11,1075.6,388.7,2221.0,able,48,185.4,93.0,H,177
17,1006.9,387.3,2051.8,able,45,185.4,73.9,L,389
24,1334.1,523.9,2841.8,able,84,172.7,73.0,L,488
37,1849.6,754.6,3999.7,able,79,172.7,65.8,L,724
...,...,...,...,...,...,...,...,...,...
65125,1164.4,450.6,2711.6,able,44,193.0,99.8,H,1465969
65272,1463.0,632.6,3418.2,able,43,165.1,89.8,H,1467609
65299,1364.3,505.0,2828.1,able,58,162.6,59.0,L,1467928
65330,1215.6,458.9,2531.2,able,45,182.9,92.5,H,1468172


# Predictor

Create a regression model to predict 2000m time

## Visualise the Relationship Between the Workouts

In [17]:
fig = px.scatter_3d(pp_5_2_10.df, x="time_2000", y="time_5000", z="time_10000", title="2000m vs 5000m vs 10000m")

# fig.update_layout(
#     scene = dict(
#         xaxis = dict(nticks=4, range=[pp_5_2_10.df["time_2000"].min(),900],),
#         yaxis = dict(nticks=4, range=[pp_5_2_10.df["time_5000"].min(),2500],),
#         zaxis = dict(nticks=4, range=[pp_5_2_10.df["time_10000"].min(),5000],)))

fig.show()

Let's try dropping some of these outlier times. Likely errors entering data or non-maximal workouts

In [18]:
#Drop 2km times above 12 minutes (720seconds)
pp_5_2_10.df = pp_5_2_10.df[pp_5_2_10.df["time_2000"] < 720]

#Drop 5km times above 38 minutes (2280 seconds)
pp_5_2_10.df = pp_5_2_10.df[pp_5_2_10.df["time_5000"] < 2280]

#Drop 5km times above 72 minutes (4320 seconds)
pp_5_2_10.df = pp_5_2_10.df[pp_5_2_10.df["time_10000"] < 4320]

In [19]:
fig = px.scatter_3d(pp_5_2_10.df, x="time_2000", y="time_5000", z="time_10000", title="2000m vs 5000m vs 10000m")

# fig.update_layout(
#     scene = dict(
#         xaxis = dict(nticks=4, range=[pp_5_2_10.df["time_2000"].min(),900],),
#         yaxis = dict(nticks=4, range=[pp_5_2_10.df["time_5000"].min(),2500],),
#         zaxis = dict(nticks=4, range=[pp_5_2_10.df["time_10000"].min(),5000],)))

fig.show()

In [20]:
fig = px.scatter_3d(pp_5_2_10.df, x="time_2000", y="time_5000", z="time_10000", title="2000m vs 5000m vs 10000m")

## Split Data

Need a train, validation and test set.

70% training data. 15% test and 15% validate

In [21]:
pp_5_2_10.split_data()

print(len(pp_5_2_10.df_train))
print(len(pp_5_2_10.df_test))
print(len(pp_5_2_10.df_val))

4968
1065
1065


## Plot Linear Regression

In [22]:
import statsmodels.formula.api as smf

y_col = "time_2000"
formula = pp.create_linear_formula(y_col,pp_5_2_10.df_train.loc[:, pp_5_2_10.df_train.columns != y_col])

model = smf.ols(formula=formula, data=pp_5_2_10.df_train).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              time_2000   R-squared:                       0.812
Model:                            OLS   Adj. R-squared:                  0.812
Method:                 Least Squares   F-statistic:                     1649.
Date:                Mon, 29 Mar 2021   Prob (F-statistic):               0.00
Time:                        16:26:45   Log-Likelihood:                -22734.
No. Observations:                4968   AIC:                         4.550e+04
Df Residuals:                    4954   BIC:                         4.559e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


p-values of all the adaptive columns are high (>0.05). Likely not enough example of adaptive athletes to be useful, let's remove these for now.

In [23]:
y_col = "time_2000"
formula = pp.create_linear_formula(y_col,["time_5000", "time_10000", "age", "height", "weight", "weight_class"])

model = smf.ols(formula=formula, data=pp_5_2_10.df_train).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              time_2000   R-squared:                       0.812
Model:                            OLS   Adj. R-squared:                  0.812
Method:                 Least Squares   F-statistic:                     3566.
Date:                Mon, 29 Mar 2021   Prob (F-statistic):               0.00
Time:                        16:26:45   Log-Likelihood:                -22740.
No. Observations:                4968   AIC:                         4.549e+04
Df Residuals:                    4961   BIC:                         4.554e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            92.1659     10.72

Weight and weight class has a a p > 0.05. Highly correlated with weight, remove.

In [24]:
y_col = "time_2000"
formula = pp.create_linear_formula(y_col,["time_5000", "time_10000", "age", "height", "weight"])

linear_model = smf.ols(formula=formula, data=pp_5_2_10.df_train).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              time_2000   R-squared:                       0.812
Model:                            OLS   Adj. R-squared:                  0.812
Method:                 Least Squares   F-statistic:                     3566.
Date:                Mon, 29 Mar 2021   Prob (F-statistic):               0.00
Time:                        16:26:46   Log-Likelihood:                -22740.
No. Observations:                4968   AIC:                         4.549e+04
Df Residuals:                    4961   BIC:                         4.554e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            92.1659     10.72

All p-values now below 0.05.

Interpreting this:
1. The heavier you are the faster you row - makes sense
2. The shorter you are, the faster you row - hmmmm
3. There is a non-zero intercept - this should probably be zero if we were just considering the relationship between different distances, addition of other parameters makes this difficult to interpret
5. The older you are, the faster you row - this could go either way
6. If you are faster at 5k and 10k ergs, your 2k will be faster - yep

### Testing and Visualising the Model

In [25]:
pp_5_2_10.df_test["time_2000_pred"] = linear_model.predict(pp_5_2_10.df_test[["time_5000", "time_10000", "age", "height", "weight"]])
pp_5_2_10.df_test["squared_error"] = (pp_5_2_10.df_test["time_2000"] - pp_5_2_10.df_test["time_2000_pred"])**2
display(pp_5_2_10.df_test)
mse = pp_5_2_10.df_test["squared_error"].mean()
print(f"Mean squared error: {round(mse,1)}s")
print(f"Mean error: {round(np.sqrt(mse),1)}s")

,time_5000,time_2000,time_10000,adaptive_rowing_category,age,height,weight,weight_class,profile_id,time_2000_pred,squared_error
0,1309.1,490.6,2597.9,able,45,162.6,61.7,L,1371976,490.332519,0.071546
1,1164.5,430.5,2542.6,able,46,180.3,90.7,H,1147040,451.714192,450.041953
2,1133.3,416.6,2502.9,able,21,177.8,68.0,L,1231580,430.141037,183.359680
3,1271.7,476.6,2587.6,able,45,182.9,107.0,H,1330927,476.363556,0.055906
4,1224.0,521.4,2603.4,able,53,180.3,98.4,H,1025375,471.797927,2460.365613
...,...,...,...,...,...,...,...,...,...,...,...
1060,1377.9,519.5,3213.5,able,54,188.0,90.7,H,1315165,538.386006,356.681222
1061,1097.9,400.1,2192.9,able,48,188.0,94.8,H,1209375,414.788987,215.766341
1062,1296.8,492.6,2573.9,able,73,182.9,70.3,L,1399679,492.948133,0.121196
1063,1195.3,441.2,2578.6,able,40,175.3,75.8,H,1147224,458.567741,301.638425


Mean squared error: 510.3s
Mean error: 22.6s


In [26]:
pp_5_2_10.df_test["residuals"] = linear_model.resid
line_y = [0] * len(pp_5_2_10.df_train["time_2000"])

pp.plot_scatter_and_line(pp_5_2_10.df_test["time_2000"], pp_5_2_10.df_test["residuals"], line_y, "Model Residuals", "y=0", "Residuals", "2000m Time", "Residuals")

Definite pattern in the residuals plot, try another model

## Linear Regression with Polynomial Terms

### 2nd Order

In [27]:
y_col = "time_2000"
x_cols = ["time_5000", "time_10000", "age", "height", "weight"]
# formula = pp.create_linear_formula(y_col,x_cols)

for df in [pp_5_2_10.df_train, pp_5_2_10.df_test, pp_5_2_10.df_val]:
    for x_col in x_cols:
        df[f"{x_col}2"] = df[f"{x_col}"]**2

formula = f"{y_col} ~ "
for x_col in x_cols:
    formula += f"{x_col} + {x_col}2 + "

formula = formula[:-3]
formula

'time_2000 ~ time_5000 + time_50002 + time_10000 + time_100002 + age + age2 + height + height2 + weight + weight2'

In [28]:
squared_model = smf.ols(formula=formula, data=pp_5_2_10.df_train).fit()
print(squared_model.summary())

                            OLS Regression Results                            
Dep. Variable:              time_2000   R-squared:                       0.820
Model:                            OLS   Adj. R-squared:                  0.820
Method:                 Least Squares   F-statistic:                     2261.
Date:                Mon, 29 Mar 2021   Prob (F-statistic):               0.00
Time:                        16:26:46   Log-Likelihood:                -22627.
No. Observations:                4968   AIC:                         4.528e+04
Df Residuals:                    4957   BIC:                         4.535e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      25.4576     75.360      0.338      

remove age2

In [29]:
formula = "time_2000 ~ time_5000 + time_50002 + time_10000 + time_100002 + age + height + height2 + weight + weight2"
squared_model = smf.ols(formula=formula, data=pp_5_2_10.df_train).fit()
print(squared_model.summary())

                            OLS Regression Results                            
Dep. Variable:              time_2000   R-squared:                       0.820
Model:                            OLS   Adj. R-squared:                  0.820
Method:                 Least Squares   F-statistic:                     2513.
Date:                Mon, 29 Mar 2021   Prob (F-statistic):               0.00
Time:                        16:26:46   Log-Likelihood:                -22627.
No. Observations:                4968   AIC:                         4.527e+04
Df Residuals:                    4958   BIC:                         4.534e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      22.1758     75.130      0.295      

Remove intercept

In [30]:
formula = "time_2000 ~ time_5000 + time_50002 + time_10000 + time_100002 + age + height + height2 + weight + weight2 -1"
squared_model = smf.ols(formula=formula, data=pp_5_2_10.df_train).fit()
print(squared_model.summary())

                                 OLS Regression Results                                
Dep. Variable:              time_2000   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                          2.329e+05
Date:                Mon, 29 Mar 2021   Prob (F-statistic):                        0.00
Time:                        16:26:46   Log-Likelihood:                         -22627.
No. Observations:                4968   AIC:                                  4.527e+04
Df Residuals:                    4959   BIC:                                  4.533e+04
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

remove time_10000

In [31]:
formula = "time_2000 ~ time_5000 + time_50002 + time_100002 + age + height + height2 + weight + weight2 -1"
squared_model = smf.ols(formula=formula, data=pp_5_2_10.df_train).fit()
print(squared_model.summary())

                                 OLS Regression Results                                
Dep. Variable:              time_2000   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                          2.620e+05
Date:                Mon, 29 Mar 2021   Prob (F-statistic):                        0.00
Time:                        16:26:46   Log-Likelihood:                         -22628.
No. Observations:                4968   AIC:                                  4.527e+04
Df Residuals:                    4960   BIC:                                  4.532e+04
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

All p now below 0.05

In [32]:
pp_5_2_10.df_test["time_2000_pred_sq"] = squared_model.predict(pp_5_2_10.df_test[["time_5000", "time_50002", "time_100002", "age", "height", "height2", "weight", "weight2"]])
pp_5_2_10.df_test["squared_error_sq"] = (pp_5_2_10.df_test["time_2000"] - pp_5_2_10.df_test["time_2000_pred_sq"])**2
display(pp_5_2_10.df_test)
mse = pp_5_2_10.df_test["squared_error_sq"].mean()
print(f"Mean squared error: {round(mse,1)}s")
print(f"Mean error: {round(np.sqrt(mse),1)}s")

,time_5000,time_2000,time_10000,adaptive_rowing_category,age,height,weight,weight_class,profile_id,time_2000_pred,squared_error,residuals,time_50002,time_100002,age2,height2,weight2,time_2000_pred_sq,squared_error_sq
0,1309.1,490.6,2597.9,able,45,162.6,61.7,L,1371976,490.332519,0.071546,1.674651,1713742.81,6749084.41,2025,26438.76,3806.89,496.370598,33.299803
1,1164.5,430.5,2542.6,able,46,180.3,90.7,H,1147040,451.714192,450.041953,13.430720,1356060.25,6464814.76,2116,32508.09,8226.49,448.767564,333.703899
2,1133.3,416.6,2502.9,able,21,177.8,68.0,L,1231580,430.141037,183.359680,129.883754,1284368.89,6264508.41,441,31612.84,4624.00,428.024696,130.523687
3,1271.7,476.6,2587.6,able,45,182.9,107.0,H,1330927,476.363556,0.055906,-5.712208,1617220.89,6695673.76,2025,33452.41,11449.00,479.904523,10.919874
4,1224.0,521.4,2603.4,able,53,180.3,98.4,H,1025375,471.797927,2460.365613,-1.965856,1498176.00,6777691.56,2809,32508.09,9682.56,471.765599,2463.573810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1377.9,519.5,3213.5,able,54,188.0,90.7,H,1315165,538.386006,356.681222,-17.670155,1898608.41,10326582.25,2916,35344.00,8226.49,539.653247,406.153350
1061,1097.9,400.1,2192.9,able,48,188.0,94.8,H,1209375,414.788987,215.766341,-60.401747,1205384.41,4808810.41,2304,35344.00,8987.04,412.831707,162.096360
1062,1296.8,492.6,2573.9,able,73,182.9,70.3,L,1399679,492.948133,0.121196,-10.006067,1681690.24,6624961.21,5329,33452.41,4942.09,497.362414,22.680585
1063,1195.3,441.2,2578.6,able,40,175.3,75.8,H,1147224,458.567741,301.638425,-10.904499,1428742.09,6649177.96,1600,30730.09,5745.64,457.958019,280.831212


Mean squared error: 503.6s
Mean error: 22.4s


In [33]:
pp_5_2_10.df_test["residuals_sq"] = squared_model.resid
line_y = [0] * len(pp_5_2_10.df_train["time_2000"])

pp.plot_scatter_and_line(pp_5_2_10.df_test["time_2000"], pp_5_2_10.df_test["residuals_sq"], line_y, "Model Residuals", "y=0", "Linear Model Residuals", "2000m Time", "Residuals")

In [34]:
pp.plot_scatter_and_line(pp_5_2_10.df_test["time_2000"], pp_5_2_10.df_test["residuals"], line_y, "Model Residuals", "y=0", "2nd Order Model Residuals", "2000m Time", "Residuals")

## Refining Model

10k^2 coefficient very small (10^-6), already reoved linear term, how does the model perform with just the 5k data?


In [35]:
formula = "time_2000 ~ time_5000 + time_50002 + age + height + height2 + weight + weight2 -1"
squared_model_5k = smf.ols(formula=formula, data=pp_5_2_10.df_train).fit()
print(squared_model_5k.summary())

                                 OLS Regression Results                                
Dep. Variable:              time_2000   R-squared (uncentered):                   0.997
Model:                            OLS   Adj. R-squared (uncentered):              0.997
Method:                 Least Squares   F-statistic:                          2.739e+05
Date:                Mon, 29 Mar 2021   Prob (F-statistic):                        0.00
Time:                        16:26:47   Log-Likelihood:                         -22850.
No. Observations:                4968   AIC:                                  4.571e+04
Df Residuals:                    4961   BIC:                                  4.576e+04
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [36]:
pp_5_2_10.df_test["time_2000_pred_sq"] = squared_model_5k.predict(pp_5_2_10.df_test[["time_5000", "time_50002", "age", "height", "height2", "weight", "weight2"]])
pp_5_2_10.df_test["squared_error_sq"] = (pp_5_2_10.df_test["time_2000"] - pp_5_2_10.df_test["time_2000_pred_sq"])**2
display(pp_5_2_10.df_test)
mse = pp_5_2_10.df_test["squared_error_sq"].mean()
print(f"Mean squared error: {round(mse,1)}s")
print(f"Mean error: {round(np.sqrt(mse),1)}s")

,time_5000,time_2000,time_10000,adaptive_rowing_category,age,height,weight,weight_class,profile_id,time_2000_pred,squared_error,residuals,time_50002,time_100002,age2,height2,weight2,time_2000_pred_sq,squared_error_sq,residuals_sq
0,1309.1,490.6,2597.9,able,45,162.6,61.7,L,1371976,490.332519,0.071546,1.674651,1713742.81,6749084.41,2025,26438.76,3806.89,505.111360,210.579566,4.930555
1,1164.5,430.5,2542.6,able,46,180.3,90.7,H,1147040,451.714192,450.041953,13.430720,1356060.25,6464814.76,2116,32508.09,8226.49,444.074602,184.269828,12.588293
2,1133.3,416.6,2502.9,able,21,177.8,68.0,L,1231580,430.141037,183.359680,129.883754,1284368.89,6264508.41,441,31612.84,4624.00,423.530892,48.037271,128.583338
3,1271.7,476.6,2587.6,able,45,182.9,107.0,H,1330927,476.363556,0.055906,-5.712208,1617220.89,6695673.76,2025,33452.41,11449.00,483.530295,48.028993,-2.701023
4,1224.0,521.4,2603.4,able,53,180.3,98.4,H,1025375,471.797927,2460.365613,-1.965856,1498176.00,6777691.56,2809,32508.09,9682.56,469.698111,2673.085355,-5.424417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1377.9,519.5,3213.5,able,54,188.0,90.7,H,1315165,538.386006,356.681222,-17.670155,1898608.41,10326582.25,2916,35344.00,8226.49,518.929948,0.324959,-19.469771
1061,1097.9,400.1,2192.9,able,48,188.0,94.8,H,1209375,414.788987,215.766341,-60.401747,1205384.41,4808810.41,2304,35344.00,8987.04,416.758301,277.499003,-62.733685
1062,1296.8,492.6,2573.9,able,73,182.9,70.3,L,1399679,492.948133,0.121196,-10.006067,1681690.24,6624961.21,5329,33452.41,4942.09,503.176684,111.866236,-13.536494
1063,1195.3,441.2,2578.6,able,40,175.3,75.8,H,1147224,458.567741,301.638425,-10.904499,1428742.09,6649177.96,1600,30730.09,5745.64,455.227524,196.771436,-9.216333


Mean squared error: 523.2s
Mean error: 22.9s


Got about as much done as I can manually, need an automated method for exploring these.

## Linear Regression Using Just 5000m time as predictor

In [37]:
models = {}

formula = "time_2000 ~ time_5000 - 1"
models["linear_5k_only"] = smf.ols(formula=formula, data=pp_5_2_10.df_train).fit()
print(models["linear_5k_only"].summary())

                                 OLS Regression Results                                
Dep. Variable:              time_2000   R-squared (uncentered):                   0.997
Model:                            OLS   Adj. R-squared (uncentered):              0.997
Method:                 Least Squares   F-statistic:                          1.674e+06
Date:                Mon, 29 Mar 2021   Prob (F-statistic):                        0.00
Time:                        16:26:47   Log-Likelihood:                         -23188.
No. Observations:                4968   AIC:                                  4.638e+04
Df Residuals:                    4967   BIC:                                  4.639e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [38]:
pp_5_2_10.df_test["time_2000_pred_sq"] = models["linear_5k_only"].predict(pp_5_2_10.df_test[["time_5000"]])
pp_5_2_10.df_test["squared_error_sq"] = (pp_5_2_10.df_test["time_2000"] - pp_5_2_10.df_test["time_2000_pred_sq"])**2
display(pp_5_2_10.df_test)
mse = pp_5_2_10.df_test["squared_error_sq"].mean()
print(f"Mean squared error: {round(mse,1)}s")
print(f"Mean error: {round(np.sqrt(mse),1)}s")

,time_5000,time_2000,time_10000,adaptive_rowing_category,age,height,weight,weight_class,profile_id,time_2000_pred,squared_error,residuals,time_50002,time_100002,age2,height2,weight2,time_2000_pred_sq,squared_error_sq,residuals_sq
0,1309.1,490.6,2597.9,able,45,162.6,61.7,L,1371976,490.332519,0.071546,1.674651,1713742.81,6749084.41,2025,26438.76,3806.89,496.235807,31.762322,4.930555
1,1164.5,430.5,2542.6,able,46,180.3,90.7,H,1147040,451.714192,450.041953,13.430720,1356060.25,6464814.76,2116,32508.09,8226.49,441.422808,119.307725,12.588293
2,1133.3,416.6,2502.9,able,21,177.8,68.0,L,1231580,430.141037,183.359680,129.883754,1284368.89,6264508.41,441,31612.84,4624.00,429.595936,168.894360,128.583338
3,1271.7,476.6,2587.6,able,45,182.9,107.0,H,1330927,476.363556,0.055906,-5.712208,1617220.89,6695673.76,2025,33452.41,11449.00,482.058724,29.797670,-2.701023
4,1224.0,521.4,2603.4,able,53,180.3,98.4,H,1025375,471.797927,2460.365613,-1.965856,1498176.00,6777691.56,2809,32508.09,9682.56,463.977258,3297.371347,-5.424417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1377.9,519.5,3213.5,able,54,188.0,90.7,H,1315165,538.386006,356.681222,-17.670155,1898608.41,10326582.25,2916,35344.00,8226.49,522.315575,7.927460,-19.469771
1061,1097.9,400.1,2192.9,able,48,188.0,94.8,H,1209375,414.788987,215.766341,-60.401747,1205384.41,4808810.41,2304,35344.00,8987.04,416.176986,258.469485,-62.733685
1062,1296.8,492.6,2573.9,able,73,182.9,70.3,L,1399679,492.948133,0.121196,-10.006067,1681690.24,6624961.21,5329,33452.41,4942.09,491.573291,1.054132,-13.536494
1063,1195.3,441.2,2578.6,able,40,175.3,75.8,H,1147224,458.567741,301.638425,-10.904499,1428742.09,6649177.96,1600,30730.09,5745.64,453.098052,141.563648,-9.216333


Mean squared error: 560.5s
Mean error: 23.7s


In [39]:
line_y = [0.3791] * pp_5_2_10.df_test["time_5000"]

pp.plot_scatter_and_line(pp_5_2_10.df_test["time_5000"], pp_5_2_10.df_test["time_2000"], line_y, "Time", "Model", "5k Time vs 2k Time", "5000m Time", "2000m Tme")

## 2nd Order Regression Using Just 5000m time as predictor

In [40]:
formula = "time_2000 ~ time_5000 + time_50002 -1"
models["squared_5k_only"] = smf.ols(formula=formula, data=pp_5_2_10.df_train).fit()
print(models["squared_5k_only"].summary())

                                 OLS Regression Results                                
Dep. Variable:              time_2000   R-squared (uncentered):                   0.997
Model:                            OLS   Adj. R-squared (uncentered):              0.997
Method:                 Least Squares   F-statistic:                          8.697e+05
Date:                Mon, 29 Mar 2021   Prob (F-statistic):                        0.00
Time:                        16:26:47   Log-Likelihood:                         -23093.
No. Observations:                4968   AIC:                                  4.619e+04
Df Residuals:                    4966   BIC:                                  4.620e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [41]:
pp_5_2_10.df_test["time_2000_pred_sq"] = models["squared_5k_only"].predict(pp_5_2_10.df_test[["time_5000", "time_50002"]])
pp_5_2_10.df_test["squared_error_sq"] = (pp_5_2_10.df_test["time_2000"] - pp_5_2_10.df_test["time_2000_pred_sq"])**2
display(pp_5_2_10.df_test)


,time_5000,time_2000,time_10000,adaptive_rowing_category,age,height,weight,weight_class,profile_id,time_2000_pred,squared_error,residuals,time_50002,time_100002,age2,height2,weight2,time_2000_pred_sq,squared_error_sq,residuals_sq
0,1309.1,490.6,2597.9,able,45,162.6,61.7,L,1371976,490.332519,0.071546,1.674651,1713742.81,6749084.41,2025,26438.76,3806.89,494.977041,19.158486,4.930555
1,1164.5,430.5,2542.6,able,46,180.3,90.7,H,1147040,451.714192,450.041953,13.430720,1356060.25,6464814.76,2116,32508.09,8226.49,444.641510,199.982298,12.588293
2,1133.3,416.6,2502.9,able,21,177.8,68.0,L,1231580,430.141037,183.359680,129.883754,1284368.89,6264508.41,441,31612.84,4624.00,433.639413,290.341598,128.583338
3,1271.7,476.6,2587.6,able,45,182.9,107.0,H,1330927,476.363556,0.055906,-5.712208,1617220.89,6695673.76,2025,33452.41,11449.00,482.061328,29.826108,-2.701023
4,1224.0,521.4,2603.4,able,53,180.3,98.4,H,1025375,471.797927,2460.365613,-1.965856,1498176.00,6777691.56,2809,32508.09,9682.56,465.484029,3126.595769,-5.424417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1377.9,519.5,3213.5,able,54,188.0,90.7,H,1315165,538.386006,356.681222,-17.670155,1898608.41,10326582.25,2916,35344.00,8226.49,518.548175,0.905971,-19.469771
1061,1097.9,400.1,2192.9,able,48,188.0,94.8,H,1209375,414.788987,215.766341,-60.401747,1205384.41,4808810.41,2304,35344.00,8987.04,421.095521,440.811907,-62.733685
1062,1296.8,492.6,2573.9,able,73,182.9,70.3,L,1399679,492.948133,0.121196,-10.006067,1681690.24,6624961.21,5329,33452.41,4942.09,490.737314,3.469598,-13.536494
1063,1195.3,441.2,2578.6,able,40,175.3,75.8,H,1147224,458.567741,301.638425,-10.904499,1428742.09,6649177.96,1600,30730.09,5745.64,455.453354,203.158091,-9.216333


In [42]:
mse = pp_5_2_10.df_test["squared_error_sq"].mean()
print(f"Mean squared error: {round(mse,1)}s")
print(f"Mean error: {round(np.sqrt(mse),1)}s")

Mean squared error: 553.5s
Mean error: 23.5s


In [43]:
pp_5_2_10.df_test = pp_5_2_10.df_test.sort_values(by="time_5000")
line_y = ([0.4118] * pp_5_2_10.df_test["time_5000"]) + ([-0.00002576] * pp_5_2_10.df_test["time_50002"])

pp.plot_scatter_and_line(pp_5_2_10.df_test["time_5000"], pp_5_2_10.df_test["time_2000"], line_y, "Time", "Model", "5k Time vs 2k Time", "5000m Time", "2000m Tme")